<a href="https://colab.research.google.com/github/Ali-mohammadi-design/LLM_Engineering_and_Machine_Learning/blob/main/LLM_Engineering_Designing_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Agents have ReAct logic. Which is based on Reasoning and Acting. They would first think about it and then would start an act then observe the act and modify it if it is required.

In [1]:
!pip install openai

In [2]:
!pip install python-dotenv

In [3]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

Note: why did we use _ in _ = load_dotenv()? When you don't care about the value being returned or assigned, you can use _ to ignore it.Thus it means that just load the envs and do not return anything.

In [4]:
from openai import OpenAI

In [6]:
import os
openAI_API_Key=os.environ['openAI_API_Key']

In [7]:
client=OpenAI(api_key=openAI_API_Key)

In [8]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [9]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [10]:
class Agent:
  def __init__(self,system=""):
    self.system=system
    self.messages=[]
    if self.system:
      self.messages.append({"role":"system","content":system})

  def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
  def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-3.5-turbo",
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content




Note: The __init__ method provides a default value for the system parameter ("", an empty string). This means you can create an instance of Agent without explicitly providing a system argument, in which case system will default to an empty string.

Note:In Python, the presence of double underscores before and after a method name (e.g., __init__, __call__) signifies that the method is a special or "magic" method. These special methods are part of Python's data model and provide a way to define the behavior of objects for specific operations. the special methods would be automatically called however, the regular method should be manually called to be used.

The strip() method is used to remove any leading and trailing whitespace (including newlines) from the multi-line string. This is helpful to ensure that the prompt variable does not start or end with unintended blank lines or spaces.

In [11]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [12]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

result = eval("4 + 5 * 2")
print(result)  # Output: 14

x = 10
result = eval("x + 5")
print(result)  # Output: 15

In [13]:
abot=Agent(prompt)

In [14]:
result=abot("how does a Toy Poodle weight?")
print(result)

Thought: I should look up the average weight of a Toy Poodle using the function average_dog_weight.

Action: average_dog_weight: Toy Poodle
PAUSE


In [15]:
result=average_dog_weight('Toy Poodle')
print(result)

a toy poodles average weight is 7 lbs


In [16]:
"observation {}:".format(result)

'observation a toy poodles average weight is 7 lbs:'

In [17]:
next_prompt="observation {}:".format(result)

In [18]:
abot(next_prompt)

'Answer: A Toy Poodle weighs 7 lbs'

In [19]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'how does a Toy P

In [20]:
abot = Agent(prompt)

In [21]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I can find the average weight of a Border Collie and a Scottish Terrier using the average_dog_weight action, then calculate their combined weight.\n\nAction: average_dog_weight: Border Collie\nPAUSE'

In [22]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [23]:
abot(next_prompt)

'Action: average_dog_weight: Scottish Terrier\nPAUSE'

In [24]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [25]:
abot(next_prompt)

'Action: calculate: 37 + 20\nPAUSE'

In [26]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [27]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs'

In [28]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user',
  'content': 'I have 2 dogs,

Note: We can do all these in a loop!!


In [62]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

The line action_re = re.compile('^Action: (\w+): (.*)$') defines a regular expression pattern that matches strings formatted as "Action: action_name: arguments" where action_name can be any sequence of word characters (\w+) and arguments can be any characters after the colon and space. This pattern is compiled into a regular expression object (action_re), which can be used to match and extract specific components from strings that follow this format

 let's break down the regular expression ^Action: (\w+): (.*)$ step by step in simpler terms, and provide examples to illustrate how it works.

Regular Expression Explanation
Pattern Breakdown:

^: Asserts the start of a line.
Action:: Matches the literal characters "Action:" exactly.
(\w+): Matches and captures one or more word characters (\w+). Word characters include letters, digits, and underscores.
:: Matches the literal colon : character.
(.*): Matches and captures any sequence of characters (.*) after the colon and space.
$: Asserts the end of a line.
What It Matches:

This pattern is designed to match lines of text that start with "Action:", followed by an action name (which can include letters, digits, or underscores), then a colon :, and then any sequence of characters that follow.
Example Usage
Let's use Python's re module to demonstrate how this regular expression works with some example strings:

python
Copy code
import re

# Define the regular expression pattern
action_re = re.compile('^Action: (\w+): (.*)$')

# Example strings to match
example_strings = [
    "Action: calculate: 4 * 7 / 2",
    "Action: average_dog_weight: Bulldog",
    "Action: search: 'Python regular expressions'",
    "Action: open_file: 'document.txt'",
    "Question: How much does a Bulldog weigh?"
]

# Iterate over example strings and try to match the pattern
for example_string in example_strings:
    match = action_re.match(example_string)
    if match:
        action_name = match.group(1)  # Retrieve the action name
        arguments = match.group(2)    # Retrieve the arguments
        print(f"Action name: {action_name}")
        print(f"Arguments: {arguments}")
    else:
        print(f"No match found for: {example_string}")
    print()  # Print a blank line for separation

Note:
In Python, the if match: statement is used to check whether a variable match contains a truthy value. Specifically, when match is the result of matching a regular expression pattern using re.match() or similar methods, if match: checks if the pattern matched successfully.


In [29]:
import re

# Define the regular expression pattern
action_re = re.compile('^Action: (\w+): (.*)$')

# Example strings to match
example_strings = [
    "Action: calculate: 4 * 7 / 2",
    "Action: average_dog_weight: Bulldog",
    "Action: search: 'Python regular expressions'",
    "Action: open_file: 'document.txt'",
    "Question: How much does a Bulldog weigh?"
]

# Iterate over example strings and try to match the pattern
for example_string in example_strings:
    match = action_re.match(example_string)
    if match:
        action_name = match.group(1)  # Retrieve the action name
        arguments = match.group(2)    # Retrieve the arguments
        print(f"Action name: {action_name}")
        print(f"Arguments: {arguments}")
    else:
        print(f"No match found for: {example_string}")
    print()  # Print a blank line for separation


Action name: calculate
Arguments: 4 * 7 / 2

Action name: average_dog_weight
Arguments: Bulldog

Action name: search
Arguments: 'Python regular expressions'

Action name: open_file
Arguments: 'document.txt'

No match found for: Question: How much does a Bulldog weigh?



Now let's see the main code:

In [63]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [53]:
known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [66]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = []
        for a in result.split('\n'):
          match = action_re.match(a)
          if match:
            actions.append(match)
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

Note: This part of the code:


   actions = [

            action_re.match(a)

            for a in result.split('\n')

            if action_re.match(a)

        ]


  Can be rewrite in this form:


    actions = []


    for a in result.split('\n'):


    match = action_re.match(a)

    if match:

        actions.append(match)

In [67]:
query("how does a Toy Poodle weight?")

Thought: I should look up the average weight of a Toy Poodle using the average_dog_weight action.

Action: average_dog_weight: Toy Poodle
PAUSE
 -- running average_dog_weight Toy Poodle
Observation: a toy poodles average weight is 7 lbs
Answer: A Toy Poodle weighs 7 lbs


In [68]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""

In [69]:
query("""I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight""")

Thought: I can find the average weight of a Border Collie and a Scottish Terrier using the average_dog_weight action, and then calculate their combined weight.

Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs
